# Anotaçõe de aulas 

Prof. Jose Storopoli

In [2]:
using BenchmarkTools
using LinearAlgebra
using Pkg
using ANSIColoredPrinters

## Loops são rápidos!

In [5]:
function summed(a)
    result = 0
    for x in a
        result += x 
    end
    return result 
end

summed (generic function with 1 method)

In [9]:
vec_a = randn(42_000);

In [7]:
@benchmark summed($vec_a)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   96.200 μs …  4.817 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      96.300 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   108.161 μs ± 71.195 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▃▁▃▂▂▁▁▁   ▁                                                ▁
  ██████████████▇█▇▆▇▇▇▆▅▆▅▆▆▆▆▆▅▅▆▅▆▅▅▅▅▅▅▄▅▅▅▄▅▅▅▄▄▄▅▄▄▄▄▃▁▅ █
  96.2 μs       Histogram: log(frequency) by time       318 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

### Dá pra ser ainda mais rápido!

In [10]:
function sumsimd(a)
    result = zero(eltype(a))
    @simd for x in a
        result += x
    end
    return result
end

sumsimd (generic function with 1 method)

In [11]:
@benchmark sumsimd($vec_a)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.829 μs … 87.486 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.886 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.354 μs ±  2.221 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ▂ ▃▁ ▁▂▁                                                 ▁
  ████████████████▇▇▇▇▅▆▅▅▆▆▅▅▆▆▅▅▄▅▄▄▅▄▅▅▅▄▄▅▄▅▄▄▁▅▄▃▄▄▄▃▃▄ █
  4.83 μs      Histogram: log(frequency) by time     14.7 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

# Dados tabulares

In [12]:
using DataFrames, BenchmarkTools

n = 10_000

df = DataFrame(
    x=rand(["A", "B", "C", "D"], n),
    y=rand(n),
    z=randn(n),
)

@benchmark combine(groupby($df, :x), :y => median, :z => mean)

BenchmarkTools.Trial: 6021 samples with 1 evaluation.
 Range (min … max):  523.700 μs …   9.794 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     704.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   820.867 μs ± 521.653 μs  ┊ GC (mean ± σ):  4.73% ± 8.21%

  ▇▂▂█▆▄▃▂▂▂▂▁▂▁▁▁▁▁▁ ▁                                         ▁
  ████████████████████████▇▇▇▇▆▅▆▅▅▆▄▁▅▃▅▁▄▄▁▄▃▃▄▁▃▁▁▁▁▁▁▃▁▃▃▁▅ █
  524 μs        Histogram: log(frequency) by time       3.47 ms <

 Memory estimate: 466.55 KiB, allocs estimate: 467.

# Funções 

Blue Style: sempre colocar return

In [ ]:
# sintaxe completa
function f_nome(arg1, arg2)
    computacoes = manipulacoes com arg1 e arg2
    return computacoes
end

In [ ]:
# sintaxte compacta
f_nome(arg1, arg2) = manipulacoes com arg1 e arg2

## Fução com mais métodos

In [15]:
function round_number(x::Float64)
    return round(x)
end

round_number (generic function with 1 method)

In [16]:
function round_number(x::Int64)
    return x
end

round_number (generic function with 2 methods)

In [18]:
methods(round_number)

# 2 methods for generic function "round_number":
[1] round_number(x::Float64) in Main at In[15]:1
[2] round_number(x::Int64) in Main at In[16]:1

In [19]:
function round_number(x::AbstractFloat)
    return round(x)
end

round_number (generic function with 3 methods)

## Multiplos valores de retorno

In [20]:
function add_multiply(x, y)
    addition = x + y
    multiplication = x * y 
    return addition, multiplication
end

add_multiply (generic function with 1 method)

In [21]:
return_1,  return_2 = add_multiply(1, 2)

(3, 2)

In [22]:
all_returns = add_multiply(1, 2)

(3, 2)

In [27]:
@show first(all_returns)
@show all_returns[1];

first(all_returns) = 3
all_returns[1] = 3


In [28]:
@show last(all_returns)
@show all_returns[2];

last(all_returns) = 2
all_returns[2] = 2


## Uso de keywords

In [29]:
function logarithm(x::Real; base::Real=ℯ)
    return log(base, x)
end

logarithm (generic function with 1 method)

In [30]:
logarithm(10)

2.302585092994046

In [32]:
logarithm(10; base=2)

3.3219280948873626

> BlueStyle: usar `;` para separar keywords tanto na criação quanto na hora de chamar a função

## Funções anônimas

In [35]:
map(x -> x^2 , 2)

4

In [36]:
map(uppercase, "Julia")

"JULIA"

In [39]:
map(x -> uppercase(x), "Julia")

"JULIA"

# Broadcast de Operadores e Funções

Vetorização = broadcasting = `.`

In [40]:
# observar a msg de erro
[1, 2, 3] + 1

LoadError: MethodError: no method matching +(::Vector{Int64}, ::Int64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  +([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
[0m  +([91m::T[39m, ::Integer) where T<:AbstractChar at char.jl:223
[0m  ...

In [41]:
[1, 2, 3] .+ 1

3-element Vector{Int64}:
 2
 3
 4